In [3]:
#Começa aqui após ter os arquivos
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads
from shapely import wkt
from branca.colormap import linear

In [4]:
#Shapefile baixado em: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html

file_name = "2021-Q04.parquet"
df = pd.read_parquet(file_name)


df.rename(columns={'tile': 'geometry'}, inplace=True)

df['geometry'] = df['geometry'].apply(wkt.loads)

df_geo = gpd.GeoDataFrame(df, geometry='geometry')
df_geo.crs = "EPSG:4326"

df_geo.head()

FileNotFoundError: [Errno 2] No such file or directory: '2021-Q04.parquet'

In [ ]:
shapefile = gpd.read_file('BR_Municipios_2022.zip') #Tentar rodar com brasil
distritos = shapefile.to_crs(4326)
distritos.head()

: 

In [ ]:
uf_dict = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AM': 'Amazonas', 'AP': 'Amapá', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MG': 'Minas Gerais', 'MS': 'Mato Grosso do Sul', 'MT': 'Mato Grosso',
    'PA': 'Pará', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'PI': 'Piauí', 'PR': 'Paraná',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RO': 'Rondônia', 'RR': 'Roraima','RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina', 'SE': 'Sergipe', 'SP': 'São Paulo', 'TO': 'Tocantins'
}
# Criando a nova coluna com os nomes completos das UFs
distritos['NOME_UF'] = distritos['SIGLA_UF'].map(uf_dict)

: 

In [ ]:
# Realiza o join espacial
join_shapefile = gpd.sjoin(df_geo, distritos, how="inner", op='intersects')
join_shapefile.head()

: 

In [ ]:
# Supondo que join_sp é o DataFrame com seus dados
# Calcular a média de avg_d_kbps por município
media_velocidade_download_sp = join_shapefile.groupby('NM_MUN')['avg_d_kbps'].mean()

# Resetar o índice para transformar em DataFrame
media_velocidade_download_sp = media_velocidade_download_sp.reset_index()

# Converter a média de kbps para Mbps diretamente na nova coluna
media_velocidade_download_sp['avg_d_mbps'] = media_velocidade_download_sp['avg_d_kbps'] / 1000
print(media_velocidade_download_sp)
# Nome do arquivo Excel
#nome_arquivo_excel = "media_velocidade_por_municipio.xlsx"
#media_velocidade_sp.to_excel(nome_arquivo_excel, index=False)

: 

In [ ]:
media_velocidade_upload = join_shapefile.groupby('NM_MUN')['avg_u_kbps'].mean()

# Resetar o índice para transformar em DataFrame
media_velocidade_upload = media_velocidade_upload.reset_index()

# Converter a média de kbps para Mbps diretamente na nova coluna
media_velocidade_upload['avg_u_mbps'] = media_velocidade_upload['avg_u_kbps'] / 1000
print(media_velocidade_upload)

: 

In [ ]:
# Merge para associar cada município à sua média de velocidade de internet
map_data = distritos.merge(media_velocidade_download_sp, left_on='NM_MUN', right_on='NM_MUN')
map_data2 = distritos.merge(media_velocidade_upload, left_on='NM_MUN', right_on='NM_MUN')
# Exibir dataframe resultante
print(map_data.head())
#map_data.to_excel('teste.xlsx')

: 

In [ ]:
gdf = gpd.GeoDataFrame(map_data, geometry='geometry')
gdf = gdf.set_geometry('geometry')
# Agrupar por estado (SIGLA_UF) e realizar as agregações
grouped = gdf.dissolve(by='NOME_UF', aggfunc={
    'AREA_KM2': 'sum',
    'avg_d_kbps': 'mean',
    'avg_d_mbps': 'mean'
})
grouped = grouped.reset_index()
print(grouped.head())



: 

In [ ]:
df_idh = pd.read_excel('Atlas - IDH Brasileiro.xlsx')
grouped = pd.merge(grouped, df_idh, left_on='NOME_UF', right_on='Territorialidade', how='left')

: 

In [ ]:
# Criar um mapa base
m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

# Definir a paleta de cores
colormap = linear.YlOrRd_09.scale(grouped['avg_d_mbps'].min(), grouped['avg_d_mbps'].max())
colormap.caption = 'Velocidade Média de Download (Mbps)'

# Adicionar os polígonos dos estados ao mapa
choropleth = folium.Choropleth(
    geo_data=grouped,
    name='choropleth',
    data=grouped,
    columns=['NOME_UF', 'avg_d_mbps'],
    key_on='feature.properties.NOME_UF',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Velocidade Média de Internet (Mbps)'
).add_to(m)

# Agora, adicionaremos os contornos com o GeoJson, definindo explicitamente a cor da linha
for _, row in grouped.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 1
        },
        name=row['NOME_UF'],
        tooltip=folium.Tooltip(f"{row['NOME_UF']}: {row['avg_d_mbps']} Mbps")
    ).add_to(m)

# Adicionar controle de camadas
folium.LayerControl().add_to(m)

# Adicionar a legenda da paleta de cores
#m.add_child(colormap)

# Salvar o mapa em um arquivo HTML
m.save('mapa_velocidade_download.html')

# Exibir o mapa
m


: 

In [ ]:
# Criar um mapa base
m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

# Definir a paleta de cores
colormap = linear.YlOrRd_09.scale(grouped['IDHM'].min(), grouped['IDHM'].max())
colormap.caption = 'Distribuição do IDH Brasileiro'

# Adicionar os polígonos dos estados ao mapa
choropleth = folium.Choropleth(
    geo_data=grouped,
    name='choropleth',
    data=grouped,
    columns=['NOME_UF', 'IDHM'],
    key_on='feature.properties.NOME_UF',
    fill_color='RdBu',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Velocidade Média de Internet (Mbps)'
).add_to(m)

# Agora, adicionaremos os contornos com o GeoJson, definindo explicitamente a cor da linha
for _, row in grouped.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 1
        },
        name=row['NOME_UF'],
        tooltip=folium.Tooltip(f"{row['NOME_UF']}: {row['IDHM']} Indice IDHM")
    ).add_to(m)

# Adicionar controle de camadas
folium.LayerControl().add_to(m)

# Adicionar a legenda da paleta de cores
#m.add_child(colormap)

# Salvar o mapa em um arquivo HTML
m.save('mapa_velocidade_IDH.html')

# Exibir o mapa
m

: 

In [ ]:
gdf2 = gpd.GeoDataFrame(map_data2, geometry='geometry')
gdf2 = gdf2.set_geometry('geometry')
# Agrupar por estado (SIGLA_UF) e realizar as agregações
grouped2 = gdf2.dissolve(by='NOME_UF', aggfunc={
    'AREA_KM2': 'sum',
    'avg_u_mbps': 'mean'
})
grouped2 = grouped2.reset_index()
print(grouped2.head())

# Criar um mapa base
m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

# Definir a paleta de cores
colormap = linear.YlOrRd_09.scale(grouped2['avg_u_mbps'].min(), grouped2['avg_u_mbps'].max())
colormap.caption = 'Velocidade Média de Upload (Mbps)'

# Adicionar os polígonos dos estados ao mapa
choropleth = folium.Choropleth(
    geo_data=grouped2,
    name='choropleth',
    data=grouped2,
    columns=['NOME_UF', 'avg_u_mbps'],
    key_on='feature.properties.NOME_UF',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Velocidade Média de Upload (Mbps)'
).add_to(m)

# Agora, adicionaremos os contornos com o GeoJson, definindo explicitamente a cor da linha
for _, row in grouped2.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 1
        },
        name=row['NOME_UF'],
        tooltip=folium.Tooltip(f"{row['NOME_UF']}: {row['avg_u_mbps']} Mbps")
    ).add_to(m)

# Adicionar controle de camadas
folium.LayerControl().add_to(m)

# Adicionar a legenda da paleta de cores
#m.add_child(colormap)

# Salvar o mapa em um arquivo HTML
m.save('mapa_velocidade_upload.html')

# Exibir o mapa
m


: 